In [6]:
import pandas as pd
import numpy as np
from scipy import sparse

In [7]:
trainData = pd.read_csv('Data/train.csv')
trainLabels = pd.read_csv('Data/train_result.csv')

In [8]:
trainData[trainData < 0] = 0
#Réduit de 73 features
trainData = trainData.loc[:, (trainData != 0).any(axis=0)]
#Réduit de 237 features
trainData = trainData.loc[:, (trainData > 0.95).any(axis=0)]

In [59]:
class LogisiticRegression:
    def __init__(self, trainData, trainLabels):
        self.trainData = trainData
        self.trainLabels = trainLabels
        self.size = len(trainData.columns)
        self.labels = np.sort(trainLabels['Class'].unique())
        self.labelSize = self.labels.size

        self.weights = pd.DataFrame(np.zeros((self.size,self.labelSize)))
        self.bias = pd.DataFrame(np.zeros(self.labelSize))

    def sigmoid(self,df):
        result = np.zeros(len(df))

        for i,val in enumerate(df):
            result[i] = np.exp(val)/np.sum(np.exp(df))

        return result
    
    def train(self, iter, batchSize):
        #for i in range(iter):

        sampleTrain = trainData.sample(batchSize,replace=False,axis=0)
        sampleLabels = trainLabels.iloc[sampleTrain.index,:]
        #Calculer le le produit de W^T*X^T+b pour tous les echantillons
        newW = self.weights.transpose()
        newX = sampleTrain.transpose()
        wx = newW.dot(newX.to_numpy())
        gFunction = (wx + self.bias).transpose()

        #Faire le argmax(softmax(g(x))) pour tous les echantillons
        softMaxResult = pd.DataFrame(np.zeros_like(gFunction))
        for i,val in gFunction.iterrows():
            softMaxResult.iloc[i,:] = self.sigmoid(val)
        predictions = np.argmax(softMaxResult,axis=1)
        predictionLabels = [self.labels[i] for i in predictions]

        return predictions

In [60]:
logReg = LogisiticRegression(trainData,trainLabels)
logReg.train(1,32)